In [15]:
import xlrd
import os
import numpy as np
import pandas as pd
hf = pd.DataFrame({})
os.chdir("/Users/jsusser/Desktop/code/funds/")
workbook = xlrd.open_workbook("../Closed End Holdings 3.xlsx")

names = workbook.sheet_names()

for x in range(1,workbook.nsheets):
    sheet = workbook.sheet_by_index(x)
    df = pd.DataFrame([sheet.row_values(x) for x in range(1, sheet.nrows)])
    df.columns = df.iloc[0]
    df = df[1:]
    name = names[x]
    if name != "ead":
        df.to_csv(name+".csv")

In [18]:
    
hf = pd.DataFrame({})
my_dir = os.listdir()
my_dir.remove(".DS_Store")
for x in my_dir:
    df = pd.read_csv(x)
    name = x.split(".csv")[0]
    cf = df["ID.WEIGHTS"].to_frame(name)
    cf.index = df.iloc[:,-1].replace(np.nan, "N/A")
    cf = cf.dropna(axis=0)
    na = cf[cf.index=="N/A"].astype('float').sum(axis=0)
    na += cf.iloc[cf.index.str[:4]=="#N/A"].sum(axis=0)
    cf = cf[(cf.index != "N/A") & (cf.index.str[:4]!="#N/A")]
    cf.loc["N/A"] = na
    cf = cf.groupby(level=0).sum()
    hf = hf.merge(cf, how='outer', left_index=True, right_index=True)
    

hf = hf.fillna(0)

#hf.index = pd.Series(hf.index).replace(np.nan, "N/A")
#hf = hf.reindex(columns=["aod","adx","peo", "aio"])
hf = hf.astype('float')

sf = pd.read_csv("../s&p500.csv")
sf = sf[["Symbol","Weight"]]
#https://www.slickcharts.com/sp500
hf

,stk,adx,chn,mie,szc,cii,grf,bcx,crf,cen,...,bmez,cet,utf,boe,igr,peo,dnp,rqi,aef,bui
Unnamed: 11,,,,,,,,,,,,,,,,,,,,,
000333,0.0,0.0,2.517473,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,1.078116,0.0
000858,0.0,0.0,1.505426,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
002179,0.0,0.0,1.058015,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
002371,0.0,0.0,1.292035,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
002475,0.0,0.0,1.062105,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNTL,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.247039,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
ZS,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.000000,0.29096,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
ZTS,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.52491,0.0,...,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [19]:
funds = hf.columns
df = pd.read_csv("../Discounts.csv", index_col=0)
my_dir = os.listdir()
my_dir.remove(".DS_Store")
funds = list(map(lambda x: x.split(".")[0].upper(), my_dir))
df = df[df.index.isin(funds)]
df = df.iloc[:,1:3]
print(list(filter(lambda x: x not in list(df.index), funds)))

df["Discount"] = list(map(lambda x: str(x).strip("%"), list(df["Discount"])))

df["52W Discount"] = [0 if x=="--" else str(x).strip('%') for x in list(df["52W Discount"])]
df = df.astype('float')
df["Discount_Weights"] = ((df["Discount"] + (df["Discount"] - df["52W Discount"]))+100)/100
discount_df = df[['Discount_Weights']]
discount_df = df.reindex([x.upper() for x in hf.columns])
discount_df

[]


,Discount,52W Discount,Discount_Weights
Ticker,,,
STK,-1.46,2.25,0.9483
ADX,-13.23,-13.68,0.8722
CHN,-10.47,-13.74,0.9280
MIE,-15.76,-11.24,0.7972
SZC,-23.64,-21.13,0.7385
CII,-9.05,-8.19,0.9009
GRF,-20.69,-18.63,0.7725
BCX,-15.15,-15.23,0.8493
CRF,18.83,9.31,1.2835


In [ ]:
listt=[]
def get_nums(num,pre_nums=[]):
    if num == 1:
        h = round(1 - sum(pre_nums),2)
        p = pre_nums + [h]
        listt.append(p)
        return p
    left = int(101 - (sum(pre_nums)*100))
    for x in range(0,left):
        pre_nums.append(x/100)

        get_nums(num-1,pre_nums)
        pre_nums = pre_nums[:0-num+1]
        
    return listt
listt=[]

hey = get_nums(len(hf.columns))
hey

In [ ]:
listt=[]
def get_nums(num,pre_nums=[]):
    if num == 1:
        h = round(1 - sum(pre_nums),1)
        p = pre_nums + [h]
        listt.append(p)
        return p
    left = int(11 - (sum(pre_nums)*10))
    for x in range(0,left):
        pre_nums.append(x/10)

        get_nums(num-1,pre_nums)
        pre_nums = pre_nums[:0-num+1]
        
    return listt
hey = get_nums(6)
len(hey)

In [ ]:
def find(weights, hf, sf):

    df = hf.multiply(weights).sum(axis=1).to_frame("ours")
    df = df.merge(sf, how="outer", right_on='Symbol', left_index=True)
    df = df.set_index("Symbol")
    #df = df.drop(["Symbol"], axis=1)
    df = df.fillna(0)
    val = sum(discount_df['Discount_Weights'] * weights)
    close_val = sum(abs(df["ours"]-df["Weight"]))
    print(round(close_val * val,2), round(close_val,2))
    return sum(abs(df["ours"]-df["Weight"]))*val

    

    
def start():
    k = 1000 
    for weights in hey:
        h = find(weights, hf, sf)
        if h<k:
            k=h
            l = weights
    print(k)
    print(l)
    
#print(find(weights, hf, sf))
weights = [1/len(hf.columns) for x in range(len(hf.columns))]
start()

#[0.1, 0.03, 0.82, 0.05]
